In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_path = "./code/files/enel1.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Define o limite de tamanho
    chunk_overlap=20  # Define a sobreposição entre *chunks*, se desejado
)
documents = text_splitter.split_documents(documents)

In [47]:
from langchain.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

vector_store = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retriever = create_retriever_tool(vector_store.as_retriever(),name="DOCUMENTO_ENERGIA",description="Documento da fatura de energia elétrica do cliente")

In [48]:
from pydantic import BaseModel,Field
from typing import  Literal

class EnergyCompanyResponse(BaseModel):
    periodo_referencia: str = Field(description="Periodo de referência da fatura no formato MM/YYYY")
    proxima_leitura: str = Field(description="Data da próxima leitura no formato DD/MM/YYYY")
    vencimento: str = Field(description="Data de vencimento da fatura no formato")
    bandeira_tarifaria: Literal['Vermelha','Amarela','Verde'] = Field(description="Bandeira tarifária da fatura")
    nome_distribuidora: str = Field(description="Nome da companhia distribuidora de energia")
    codigo_cliente: str = Field(description="Código do cliente na distribuidora")
    codigo_instalacao: str = Field(description="Código da instalação na distribuidora")
    tipo_fornecimento: Literal['Monofásico','Bifásico','Trifásico'] = Field(description="Tipo de fornecimento de energia")
    valor_total: float = Field(description="Valor total da fatura")
    consumo_kwh: float = Field(description="Consumo em kWh")
    client_nome: str = Field(description="Nome do cliente")
    
from langchain_core.output_parsers import JsonOutputParser
output_parser = JsonOutputParser(pydantic_object=EnergyCompanyResponse)

In [49]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [50]:
from langchain_core.prompts import  PromptTemplate
prompt = PromptTemplate(
    template="""
                Answer the following questions as best you can. 
                Think step by step to find each field of format instructions.
                You have access to the following tools: {tools}
                The final answer should be in the following format: {format_instructions}

                Use the following format:

                Question: the input question you must answer
                Thought: you should always think about what to do
                Action: the action to take, should be one of [{tool_names}]
                Action Input: the input to the action
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times)
                Thought: I now know the final answer
                Final Answer: the final answer to the original input question
                The final answer should be in the following format: {format_instructions}
                Begin!

                Question: {input}
                Thought:{agent_scratchpad}
    """,
    partial_variables={
        "format_instructions":output_parser.get_format_instructions(),
    }
)

In [51]:
from langchain.agents import create_react_agent,AgentExecutor
agent = create_react_agent(llm=llm,prompt=prompt,tools=[retriever])
agent_executor = AgentExecutor(agent=agent,tools=[retriever],verbose=True,handle_parsing_errors=True)

In [52]:
result = agent_executor.invoke({
    "input":"Quais os dados da fatura de energia do cliente?"
})



> Entering new AgentExecutor chain...
Action: DOCUMENTO_ENERGIA
Action Input: query: 'Quais os dados da fatura de energia do cliente?'28374617
27364518
DOCUMENTO AUXILIAR DA NOTA FISCAL DE ENERGIA ELÉTRICA ELETRÔNICA
A Tarifa Social de Energia Elétrica foi criada pela lei 10.438 de 26 de abril de 2002

ENEL
Períodos: Band. Tarif.: Verde : 25/04 - 25/05 Bandeira verde em maio/23, sem custos adicionais na fatura. Informações: www.aneel.gov.br
{BARCODE}
Ampla Energia e Serviços S. A.

CFOP 5258: VENDA DE ENERGIA ELETRICA A NAO CONTRIBUINTE
Data de apresentação: 29/05/2023
 B1 RESIDENCIAL-CONV. - Residencial
Residencial TRIFÁSICO
05/2023 15/06/2023 R$ 338,45
VICTOR  NASCIMENTO

Energia Ativa Fornecida TUSD 212,74290 3,68 212,740,73359kWh 18,00% 38,29 0,58888
CIP - ILUM PUB PREF MUNICIPAL 17,54 0,00 0,00 0,00% 0,00
Subtotal Faturamento
Subtotal Outros
TOTAL 5,54 320,91 57,76Thought: The document contains the necessary information for the energy bill.
Action: Extract the relevant data from

In [53]:
print(result['output'])

{
    "periodo_referencia": "05/2023",
    "proxima_leitura": "15/06/2023",
    "vencimento": "29/05/2023",
    "bandeira_tarifaria": "Verde",
    "nome_distribuidora": "ENEL",
    "codigo_cliente": "27364518",
    "codigo_instalacao": "30921-GNE-051",
    "tipo_fornecimento": "Trifásico",
    "valor_total": 338.45,
    "consumo_kwh": 212.74,
    "client_nome": "VICTOR NASCIMENTO"
}
